In [ ]:
from ultralytics import YOLO
MODEL_PATH = r"models\focus1\Focus1_YOLO11s_x1024_14112024.pt"

# Export to ONNX

In [3]:
model = YOLO(MODEL_PATH)
model.export(format='tflite', data=r"Focus1\Focus1\new dataset_2000\batrec.v9-allen_yolo8_11112024.yolov8-obb\data.yaml", int8 = True)

Ultralytics 8.3.162  Python-3.11.9 torch-2.7.1+cu128 CPU (Intel Core(TM) Ultra 7 165H)
YOLO11s summary (fused): 100 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs

PyTorch: starting from 'models\focus1\Focus1_YOLO11s_x1024_14112024.pt' with input shape (1, 3, 1024, 1024) BCHW and output shape(s) (1, 6, 21504) (18.4 MB)
requirements: Ultralytics requirement ['ai-edge-litert>=1.2.0,<1.4.0'] not found, attempting AutoUpdate...
WARNING Retry 1/2 failed: Command 'pip install --no-cache-dir "ai-edge-litert>=1.2.0,<1.4.0" --extra-index-url https://pypi.ngc.nvidia.com' returned non-zero exit status 1.
WARNING Retry 2/2 failed: Command 'pip install --no-cache-dir "ai-edge-litert>=1.2.0,<1.4.0" --extra-index-url https://pypi.ngc.nvidia.com' returned non-zero exit status 1.
WARNING requirements:  Command 'pip install --no-cache-dir "ai-edge-litert>=1.2.0,<1.4.0" --extra-index-url https://pypi.ngc.nvidia.com' returned non-zero exit status 1.

TensorFlow SavedModel: starting export with ten

Scanning C:\Users\lin40269\Desktop\Linh\01_Python\realsense\Focus1\Focus1\new dataset_2000\batrec.v9-allen_yolo8_11112024.yolov8-obb\valid\labels.cache... 213 images, 0 backgrounds, 0 corrupt: 100%|██████████| 213/213 [00:00<?, ?it/s]

WARNING TensorFlow SavedModel: >300 images recommended for INT8 calibration, found 213 images.



ERROR TensorFlow SavedModel: export failure 46.7s: No module named 'ai_edge_litert'


ModuleNotFoundError: No module named 'ai_edge_litert'

# Quantize model

## Dynamic Quantization

In [1]:
from onnxruntime.quantization import quantize_dynamic, QuantType

quantize_dynamic(
    model_input=r"models\focus1\Focus1_YOLO11s_x1024_14112024.onnx",
    model_output=r"models\focus1\Focus1_YOLO11s_quant.onnx",
    weight_type=QuantType.QInt8  # or QUInt8
)

## Static quantization

In [14]:
from onnxruntime.quantization import CalibrationDataReader
import cv2
import numpy as np
import glob

class ImageFolderCalibrationReader(CalibrationDataReader):
    def __init__(self, image_dir, input_name, input_size=(640, 640), max_samples=10):
        self.input_name = input_name
        image_paths = glob.glob(f"{image_dir}/*.jpg")[:max_samples]
        self.data = []

        for path in image_paths:
            img = cv2.imread(path)
            img = cv2.resize(img, input_size)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = img.transpose(2, 0, 1) / 255.0
            tensor = np.expand_dims(img, axis=0).astype(np.float32)
            self.data.append({self.input_name: tensor})

        self.iterator = iter(self.data)

    def get_next(self):
        return next(self.iterator, None)


In [15]:
from onnxruntime.quantization import quantize_static, QuantFormat, QuantType

# Replace with your actual input name (e.g., "images")
input_name = "images"
calibration_reader = ImageFolderCalibrationReader(
    image_dir=r"Focus1\Focus1\new dataset_2000\batrec.v9-allen_yolo8_11112024.yolov8-obb\test\images", 
    input_name=input_name,
    input_size=(640, 640)
)

quantize_static(
    model_input=r"models\focus1\Focus1_YOLO11s_x1024_14112024.onnx",
    model_output=r"models\focus1\Focus1_YOLO11s_quant_static.onnx",
    calibration_data_reader=calibration_reader,
    quant_format=QuantFormat.QDQ,  # Use QDQ format
    weight_type=QuantType.QInt8,
    activation_type=QuantType.QInt8
)


# Test run model

## Load model

In [ ]:
import onnxruntime as ort

# Load model
session = ort.InferenceSession("yolov11.onnx", providers=["CPUExecutionProvider"])

# Get input name
input_name = session.get_inputs()[0].name



## Video inference pipeline

In [10]:
import cv2
import numpy as np
import onnxruntime as ort

def preprocess(frame, input_size=640):
    img = cv2.resize(frame, (input_size, input_size))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.transpose(2, 0, 1) / 255.0
    return img[np.newaxis, ...].astype(np.float32)

def nms(boxes, scores, iou_threshold=0.5):
    indices = cv2.dnn.NMSBoxes(boxes, scores, score_threshold=0.25, nms_threshold=iou_threshold)
    
    if indices is None or len(indices) == 0:
        return []
    
    # Flatten if needed
    if isinstance(indices, np.ndarray):
        indices = indices.flatten().tolist()
    elif isinstance(indices[0], (list, tuple)):
        indices = [i[0] for i in indices]
    
    return indices

def postprocess(preds, frame_shape, input_size=640, conf_thres=0.25, iou_thres=0.45):
    detections = []
    pred = preds[0][0]  # shape: (N, 85)

    for det in pred:
        conf = det[4]
        if conf < conf_thres:
            continue
        class_id = np.argmax(det[5:])
        score = conf * det[5:][class_id]
        x, y, w, h = det[:4]
        x1 = int((x - w / 2) / input_size * frame_shape[1])
        y1 = int((y - h / 2) / input_size * frame_shape[0])
        x2 = int((x + w / 2) / input_size * frame_shape[1])
        y2 = int((y + h / 2) / input_size * frame_shape[0])
        detections.append(([x1, y1, x2 - x1, y2 - y1], score, class_id))

    if not detections:
        return []
    
    boxes, scores, class_ids = zip(*detections)
    keep = nms(boxes, scores, iou_threshold=iou_thres)
    return [(*boxes[i], scores[i], class_ids[i]) for i in keep]


In [7]:
MODEL_PATH_INFERENCE = r"models\focus1\Focus1_YOLO11s_x1024_14112024.onnx"

session = ort.InferenceSession(MODEL_PATH_INFERENCE)
input_name = session.get_inputs()[0].name

In [12]:
import pyrealsense2 as rs
import numpy as np
import cv2

# Setup RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
pipeline.start(config)

In [13]:
try:
    while True:
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        if not color_frame:
            continue

        frame = np.asanyarray(color_frame.get_data())
        input_tensor = preprocess(frame)
        outputs = session.run(None, {input_name: input_tensor})
        detections = postprocess(outputs, frame.shape[:2])

        for x1, y1, x2, y2, score, cls in detections:
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{cls}:{score:.2f}", (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)

        cv2.imshow("YOLOv11 + RealSense", frame)
        if cv2.waitKey(1) == ord('q'):
            break

finally:
    pipeline.stop()
    cv2.destroyAllWindows()
